In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('../..')
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from exploration.config import mongo_inst
from mlpp.data_collection.sample import ScoresSubset
from mlpp.data_collection.sample_func import SampleConfig, SampleFunctionGenerator as SFG

In [ ]:
NUM_BINS = 200
MAX_PP = 7000
DATE_LIMIT = datetime(2019,1,1)
SAMPLE_CONFIG = SampleConfig(date_limit = DATE_LIMIT, max_pp = MAX_PP, n_bins = NUM_BINS)

osu_random_db = mongo_inst['osu_random_db']
osu_subset = ScoresSubset(osu_random_db['osu_scores_high'], osu_random_db['osu_user_stats'])

generator = SFG(osu_subset, SAMPLE_CONFIG)

In [ ]:
greedy_sample_func, predicted_uniformity_arr, bins_hist = generator.greedy(.01)

In [ ]:
%matplotlib notebook

bins = np.linspace(0, MAX_PP, num=NUM_BINS + 1, dtype=int)

def animate(i):
    plt.cla()
    plt.title("Greedy Sampling on decreasing PP ranges")
    plt.xlabel("Score est pp")
    plt.ylabel("Count included")
    plt.hist(bins[:-1], NUM_BINS, weights = bins_hist[i])
    plt.ylim([0, 1000])
    high = int(MAX_PP * (1 - i / NUM_BINS))
    low = int(high - MAX_PP / NUM_BINS)
    plt.gca().annotate(f'range={low}-{high}', [5000, 3800])

fig = plt.figure()

ani = animation.FuncAnimation(fig, animate, frames=NUM_BINS, save_count=NUM_BINS, repeat = False)
ani.save("greedy_sampling_progression.gif", fps = 30)